In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from torchvision import datasets, transforms
from PIL import Image
import torch
import torch.nn as nn

In [2]:
# Use GPU if available
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
device

'cuda:0'

In [3]:
from torchvision.models import resnet50, ResNet50_Weights
def get_resnet50():
    model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
    model.fc = torch.nn.Sequential(
        torch.nn.Linear(model.fc.in_features, 100),
        torch.nn.ReLU(),
        torch.nn.Linear(100, 4),
        torch.nn.Softmax()
    )
    return model

In [4]:
def finetune(model, epochs, train_dataloader, optimizer, loss_function, val_dataloader=None, checkpoint_epochs=5, max_epochs_without_imp=2):
  history = {
      "loss": {
          "train": [],
          "val": []
      },
      "accuracy": {
          "train": [],
          "val": []
      }
  }
  epochs_without_imp = 0
  for epoch in range(1, epochs+1):
    print(f"Epoch {epoch}/{epochs}", end="\t-\t")

    model.train()  # put network in train mode for Dropout and Batch Normalization
    train_loss = torch.tensor(0., device=device)  # loss and accuracy tensors are on the GPU to avoid data transfers
    train_accuracy = torch.tensor(0., device=device)
    for X, y in train_dataloader:
      X = X.to(device)
      y = y.to(device)
      ypred = model(X)
      loss = loss_function(ypred, y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      with torch.no_grad():
        train_loss += loss*train_dataloader.batch_size
        train_accuracy += (torch.argmax(ypred, dim=1) == y).sum()

    history["loss"]["train"].append(train_loss.item()/len(train_dataloader.dataset))
    history["accuracy"]["train"].append(train_accuracy.item()/len(train_dataloader.dataset))
    print(f"train loss: {history['loss']['train'][-1]:.2f}", end=", ")
    print(f"train acc: {history['accuracy']['train'][-1]:.2f}", end="\t-\t")

    if val_dataloader is not None:
      model.eval()
      val_loss = torch.tensor(0., device=device)
      val_accuracy = torch.tensor(0., device=device)
      with torch.no_grad():
        for X, y in val_dataloader:
          X = X.to(device)
          y = y.to(device)
          ypred = model(X)
          loss = loss_function(ypred, y)
          val_loss += loss * val_dataloader.batch_size
          val_accuracy += (torch.argmax(ypred, dim=1) == y).sum()

      history["loss"]["val"].append(val_loss.item()/len(val_dataloader.dataset))
      history["accuracy"]["val"].append(val_accuracy.item()/len(val_dataloader.dataset))
      print(f"val loss: {history['loss']['val'][-1]:.2f}", end=", ")
      print(f"val acc: {history['accuracy']['val'][-1]:.2f}")

    if epoch >= 2 and history["loss"]["train"][-1] >= history["loss"]["train"][-2]:
      epochs_without_imp += 1
    else:
      epochs_without_imp = 0
    if epochs_without_imp >= max_epochs_without_imp:
      break

  return model, history

## Model Evaluation

### Accuracy on the test set

In [5]:
#load the saved model
MODEL_PATH = "lettuce_npk.pth"
model = torch.load(MODEL_PATH)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

### Visualization

In [6]:
# create a mapping from class labels to class names
classes = ["-K", "-N", "-P", "FN"]
class_names = dict(zip(range(len(classes)), sorted(classes)))

### Creating a prediction function
that takes as input an image and predits its class with the corresponding confidence

In [7]:
def prediction(img):
    t = transforms.Compose([
      transforms.ToTensor(),
      transforms.Resize(224, antialias=True)
    ])
    new_img = t(img)
    model.eval()
    with torch.no_grad():
        pred = model(torch.stack([new_img]).to(device)).cpu().detach().numpy()[0]
    class_label = np.argmax(pred)
    return class_names[np.argmax(pred)], pred[class_label]

i=0
img_path = "data/k_12.png"
print(img_path)
img = Image.open(img_path)
prediction(img)

data/k_12.png


C:\Users\abdou\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


('-K', 0.9755307)

In [8]:
import gradio as gr

demo = gr.Interface(prediction, gr.Image(), "text")
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 4.16.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://4b123f9702de7dc470.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [9]:
demo = gr.Interface(prediction, gr.Image(), "text")
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.16.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://aeaf000b5c05b308f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


C:\Users\abdou\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
C:\Users\abdou\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
